In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
import sys
import haversine as hs
import os
import json
import re
from os.path import join
from pathlib import Path
from sklearn.neighbors import BallTree

In [2]:
inp_graph = "../graphfiles/"
inp_data = "./../Delhi/Delhi.csv"
out_dir = "data_22nov"
isgraph = False
isrestaurants = False
isvehicle = False
isorders = True

In [3]:
Path(out_dir).mkdir(parents=True, exist_ok=True)   
df = pd.read_csv(join(inp_graph, 'nodes_mod_connected.csv'))
l = df.to_dict('records')

nodes = df[['lat', 'lon']].values * np.pi/180.0
tree = BallTree(nodes, metric = 'haversine', leaf_size=1)
idx_to_id = df.id.values.tolist()

df2 = pd.read_csv(join(inp_graph, 'segments_mod_connected.csv'))
df2 = df2.sort_values(by=['u', 'v'])
l2 = df2.to_dict('records')

In [6]:
# assigning old node id to new to id
dic = {}
cnt = 1
for entry in l2:
    if entry['u'] not in dic:
        dic[entry['u']] = cnt
        cnt+=1
        
for entry in l2:        
    if entry['v'] not in dic:
        dic[entry['v']] = cnt
        cnt+=1

nodeidToLatLong = {dic[node['id']] : (node['lat'], node['lon']) for node in l}  # newnodeid : lat,long
graphList = [ [dic[x['u']], dic[x['v']], x['dist']] for x in l2]                # u,v,dist
len(dic), len(nodeidToLatLong), len(graphList)

(182471, 182471, 460584)

In [7]:
if isgraph:
    speed = 8
    f = open(join(out_dir, 'graph.txt'), 'w')
    f.write(f"p sp {len(dic)} {len(l2)}\n")
    for l in graphList:
        f.write(f"a {l[0]} {l[1]} {max(1, round(l[2]/speed))}\n")
    f.close()

In [3]:
df = pd.read_csv(inp_data)
print(df.shape)
df = df.drop(['order_assigned_time', 'order_saved_time'], axis=1)
df = df.drop(['order_status', 'multi_run_order','allocation_source', 'planned_delivery_distance_km', 'planned_return_hub_distance_km','actual_delivery_distance_km', 'actual_return_hub_distance_km'], axis=1)
df = df.drop(['Unnamed: 0', 'order_number', 'score', 'rider_name', 'rider_type', 'startbike_time', 'reachedgate_time', 'startbike_to_reachedgate_in_mins', 'backtostore_time', 'suspicious_reachedgate_flag', 'suspicious_delivered_flag', 'startbike_lat_lng', 'reachedgate_lat_lng', 'delivered_lat_lng', 'backtostore_lat_lng'], axis=1)

print(df.shape)


(476882, 35)
(476882, 11)


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476882 entries, 0 to 476881
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_date           476882 non-null  object 
 1   order_items          476882 non-null  object 
 2   store_id             476882 non-null  object 
 3   store_lat            476882 non-null  float64
 4   store_lng            476882 non-null  float64
 5   customer_lat         476882 non-null  float64
 6   customer_lng         476882 non-null  float64
 7   rider_id             476882 non-null  object 
 8   order_assigned_time  476868 non-null  object 
 9   delivered_time       476882 non-null  object 
 10  makeline_time        476882 non-null  object 
 11  order_saved_time     476882 non-null  object 
 12  order_received_time  476882 non-null  object 
dtypes: float64(4), object(9)
memory usage: 47.3+ MB


In [9]:
# print(df['delivered_time'].unique())
df['time'] = df['delivered_time']-df['order_received_time']
l = [x for x in df['time']]
print(len(l), min(l), max(l))

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [9]:
dfr = df.groupby(['store_id','store_lat','store_lng']).size().reset_index(name='count')
rest_latlong = dfr[['store_lat','store_lng']].values.tolist()
rest_latlong = np.array(rest_latlong).astype('float')
rest_latlong = rest_latlong * np.pi / 180.0
ans = tree.query(rest_latlong, k=1, return_distance=True, dualtree=True, breadth_first=False)
dfr["distance"] = ans[0]*6378137
dfr["node_id"] = ans[1]
dfr['node_id'] = dfr['node_id'].apply(lambda x: dic[idx_to_id[x]])

storetonodemap = dict(dfr[['store_id', 'node_id']].to_dict('split')['data'])

In [10]:
dfd = df.groupby(['rider_id','store_id']).size().reset_index(name='count')
dfd['node_id'] = dfd['store_id'].apply(lambda x: storetonodemap[x])
drivertonodemap = dict(dfd[['rider_id', 'node_id']].to_dict('split')['data'])

In [5]:
# Orders start from here
df = df.sort_values(by=['order_date', 'order_received_time'])
order_latlong = df[['customer_lat','customer_lng']].values.tolist()
order_latlong = np.array(order_latlong).astype('float')
order_latlong = order_latlong * np.pi / 180.0
ans = tree.query(order_latlong, k=1, return_distance=True, dualtree=True, breadth_first=False)
df["distance"] = ans[0]*6378137
df["node_id"] = ans[1]
df['node_id'] = df['node_id'].apply(lambda x: dic[idx_to_id[x]])

ordersDelhi = df.to_dict('records')
df.shape

NameError: name 'dic' is not defined

In [7]:
def TimeToEpoch(t):
    try:
        dt1 = datetime.datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%f")
        dt2 = dt1.replace(hour=0, minute=0, second=0)
        return abs(dt1-dt2).total_seconds()
    except:
        dt1 = datetime.datetime.strptime(t, "%Y-%m-%dT%H:%M:%S")
        dt2 = dt1.replace(hour=0, minute=0, second=0)
        return abs(dt1-dt2).total_seconds()

In [8]:
df['time'] = df['order_received_time'].apply(lambda x:round(TimeToEpoch(x)))
# df['delivered_time'] = pd.to_datetime(df['delivered_time'])
# df['makeline_time'] = pd.to_datetime(df['makeline_time'])
df['o2'] = pd.to_datetime(df['order_received_time'])
df['date'] = df['o2'].dt.date
df['slot'] = df['o2'].dt.hour

df

,order_date,order_items,store_id,store_lat,store_lng,customer_lat,customer_lng,rider_id,delivered_time,makeline_time,order_received_time,time,o2,date,slot
0,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""PIZ0140"", ""volu...",DPI65781,28.474090,77.047800,28.476961,77.057187,90341910_DPI65781,2021-12-17T23:41:25.859+05:30,2021-12-17T23:19:45+05:30,2021-12-17T23:20:05.291,84005,2021-12-17 23:20:05.291,2021-12-17,23
1,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""PIZ0133"", ""volu...",DPI66505,28.657971,77.193519,28.657608,77.201147,90324964_DPI66505,2021-12-17T23:10:36.02+05:30,2021-12-17T22:49:36+05:30,2021-12-17T22:50:11.979,82211,2021-12-17 22:50:11.979,2021-12-17,22
2,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""BEV0119"", ""volu...",DPI66105,28.693783,77.150561,28.686944,77.149429,938030_DPI66105,2021-12-17T22:55:29.452+05:30,2021-12-17T22:27:18+05:30,2021-12-17T22:27:44.776,80864,2021-12-17 22:27:44.776,2021-12-17,22
3,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""DIP0006"", ""volu...",DPI66116,28.709763,77.189156,28.705712,77.192963,90323414_DPI66116,2021-12-17T21:39:07.141+05:30,2021-12-17T21:10:17+05:30,2021-12-17T21:10:27.119,76227,2021-12-17 21:10:27.119,2021-12-17,21
4,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""BRD0020"", ""volu...",DPI65882,28.599570,77.294290,28.599363,77.298087,90360700_DPI65882,2021-12-17T20:29:45.328+05:30,2021-12-17T20:15:30+05:30,2021-12-17T20:15:39.892,72939,2021-12-17 20:15:39.892,2021-12-17,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476877,2021-12-30,"[{""quantity"": 1, ""menu_code"": ""PIZ5122"", ""volu...",DPI66718,28.631446,77.278632,28.633383,77.275763,90355273_DPI66718,2021-12-30T20:28:59.365+05:30,2021-12-30T20:00:18+05:30,2021-12-30T20:00:46.858,72046,2021-12-30 20:00:46.858,2021-12-30,20
476878,2021-12-30,"[{""quantity"": 1, ""menu_code"": ""DIP0005"", ""volu...",DPI65834,28.646950,77.118597,28.637177,77.121797,90335544_DPI65834,2021-12-30T20:28:10.201+05:30,2021-12-30T20:10:19+05:30,2021-12-30T20:10:29.446,72629,2021-12-30 20:10:29.446,2021-12-30,20
476879,2021-12-30,"[{""quantity"": 1, ""menu_code"": ""PIZ0126"", ""volu...",DPI65957,28.638470,77.360670,28.634223,77.366078,90321269_DPI65957,2021-12-30T20:24:02.92+05:30,2021-12-30T20:07:12+05:30,2021-12-30T20:07:37.635,72457,2021-12-30 20:07:37.635,2021-12-30,20
476880,2021-12-30,"[{""quantity"": 1, ""menu_code"": ""PIZ0132"", ""volu...",DPI66158,28.663046,77.281455,28.676193,77.282897,90342775_DPI66158,2021-12-30T20:41:08.969+05:30,2021-12-30T20:15:46+05:30,2021-12-30T20:15:53.945,72953,2021-12-30 20:15:53.945,2021-12-30,20


<!-- df['date'].unique().astype(str).tolist() -->

In [8]:
itemDict = {}
i = 0
for oi in df['order_items']:
    a = json.loads(oi)
    for y in a:
        l1 = [y['menu_description'], y['quantity'], y['order_line_total'], y['menu_code']]
        if y['menu_code'] not in itemDict:
            itemDict[y['menu_code']] = i
            i+=1
# itemDict

In [15]:
def getItemsList(oi):
    a = json.loads(oi)
    ret = []
    for y in a:
        l1 = [y['menu_description'], y['quantity'], y['order_line_total'], y['menu_code']]
        for j in range(0, y['quantity']):
            ret.append(itemDict[y['menu_code']])
    return ret
df['menu'] = df['order_items'].apply(getItemsList)

In [9]:
tpath = out_dir
Path(tpath).mkdir(parents=True, exist_ok=True)   
dates = df['date'].unique().astype(str).tolist()
for date in dates:
    Path(join(tpath,date)).mkdir(parents=True, exist_ok=True)
slots = [x for x in range(24)]
tpath

'data_22nov'

In [9]:
date = '2021-12-17'
slot = 11
tdf = df[(df['date'].astype(str)==date) & (df['slot'] == slot)]
tdf

,order_date,order_items,store_id,store_lat,store_lng,customer_lat,customer_lng,rider_id,delivered_time,makeline_time,order_received_time,time,o2,date,slot
2802,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""PIZ0162"", ""volu...",DPI66479,28.489530,77.066270,28.479762,77.066784,90352207_DPI66479,2021-12-17T23:06:30.913+05:30,2021-12-17T11:07:36+05:30,2021-12-17T11:07:57.067,40077,2021-12-17 11:07:57.067,2021-12-17,11
11908,2021-12-17,"[{""quantity"": 20, ""menu_code"": ""DIP0005"", ""vol...",DPI67051,28.577789,77.317464,28.574193,77.312550,90334887_DPI67051,2021-12-17T17:13:26.779+05:30,2021-12-17T11:54:50+05:30,2021-12-17T11:54:57.849,42897,2021-12-17 11:54:57.849,2021-12-17,11
13473,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""BRD0020"", ""volu...",DPI66321,28.458810,77.527900,28.464670,77.537022,90357643_DPI66321,2021-12-17T11:21:17.701+05:30,2021-12-17T11:01:09+05:30,2021-12-17T11:01:31.984,39691,2021-12-17 11:01:31.984,2021-12-17,11
13474,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""BEV0119"", ""volu...",DPI66321,28.458810,77.527900,28.483653,77.513113,90357643_DPI66321,2021-12-17T12:17:49.394+05:30,2021-12-17T11:49:22+05:30,2021-12-17T11:49:37.237,42577,2021-12-17 11:49:37.237,2021-12-17,11
13486,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""PIZ0130"", ""volu...",DPI65906,28.473250,77.515600,28.471889,77.481437,90358257_DPI65906,2021-12-17T11:45:49.023+05:30,2021-12-17T11:08:27+05:30,2021-12-17T11:08:43.605,40123,2021-12-17 11:08:43.605,2021-12-17,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125126,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""PIZ0117"", ""volu...",DPI65899,28.735930,77.113170,28.742510,77.115859,90335322_DPI65899,2021-12-17T11:22:05.069+05:30,2021-12-17T11:04:20+05:30,2021-12-17T11:04:52.46,39892,2021-12-17 11:04:52.460,2021-12-17,11
125127,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""PIZ0119"", ""volu...",DPI65908,28.677967,77.104601,28.675770,77.110650,90346559_DPI65908,2021-12-17T11:23:50.239+05:30,2021-12-17T11:03:35+05:30,2021-12-17T11:03:40.845,39820,2021-12-17 11:03:40.845,2021-12-17,11
125128,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""PIZ0134"", ""volu...",DPI65796,28.679921,77.208224,28.674266,77.204407,90321520_DPI65796,2021-12-17T11:24:06.535+05:30,2021-12-17T11:03:05+05:30,2021-12-17T11:09:12.433,40152,2021-12-17 11:09:12.433,2021-12-17,11
125129,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""PIZ0117"", ""volu...",DPI67060,28.506131,77.231868,28.506864,77.240543,90338690_DPI67060,2021-12-17T11:22:34.495+05:30,2021-12-17T11:02:49+05:30,2021-12-17T11:02:58.836,39778,2021-12-17 11:02:58.836,2021-12-17,11


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530 entries, 2802 to 125130
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   order_date           530 non-null    object        
 1   order_items          530 non-null    object        
 2   store_id             530 non-null    object        
 3   store_lat            530 non-null    float64       
 4   store_lng            530 non-null    float64       
 5   customer_lat         530 non-null    float64       
 6   customer_lng         530 non-null    float64       
 7   rider_id             530 non-null    object        
 8   delivered_time       530 non-null    object        
 9   makeline_time        530 non-null    object        
 10  order_received_time  530 non-null    object        
 11  time                 530 non-null    int64         
 12  o2                   530 non-null    datetime64[ns]
 13  date                 530 non-

In [11]:
df = tdf
df

,order_date,order_items,store_id,store_lat,store_lng,customer_lat,customer_lng,rider_id,delivered_time,makeline_time,order_received_time,time,o2,date,slot
2802,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""PIZ0162"", ""volu...",DPI66479,28.489530,77.066270,28.479762,77.066784,90352207_DPI66479,2021-12-17T23:06:30.913+05:30,2021-12-17T11:07:36+05:30,2021-12-17T11:07:57.067,40077,2021-12-17 11:07:57.067,2021-12-17,11
11908,2021-12-17,"[{""quantity"": 20, ""menu_code"": ""DIP0005"", ""vol...",DPI67051,28.577789,77.317464,28.574193,77.312550,90334887_DPI67051,2021-12-17T17:13:26.779+05:30,2021-12-17T11:54:50+05:30,2021-12-17T11:54:57.849,42897,2021-12-17 11:54:57.849,2021-12-17,11
13473,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""BRD0020"", ""volu...",DPI66321,28.458810,77.527900,28.464670,77.537022,90357643_DPI66321,2021-12-17T11:21:17.701+05:30,2021-12-17T11:01:09+05:30,2021-12-17T11:01:31.984,39691,2021-12-17 11:01:31.984,2021-12-17,11
13474,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""BEV0119"", ""volu...",DPI66321,28.458810,77.527900,28.483653,77.513113,90357643_DPI66321,2021-12-17T12:17:49.394+05:30,2021-12-17T11:49:22+05:30,2021-12-17T11:49:37.237,42577,2021-12-17 11:49:37.237,2021-12-17,11
13486,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""PIZ0130"", ""volu...",DPI65906,28.473250,77.515600,28.471889,77.481437,90358257_DPI65906,2021-12-17T11:45:49.023+05:30,2021-12-17T11:08:27+05:30,2021-12-17T11:08:43.605,40123,2021-12-17 11:08:43.605,2021-12-17,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125126,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""PIZ0117"", ""volu...",DPI65899,28.735930,77.113170,28.742510,77.115859,90335322_DPI65899,2021-12-17T11:22:05.069+05:30,2021-12-17T11:04:20+05:30,2021-12-17T11:04:52.46,39892,2021-12-17 11:04:52.460,2021-12-17,11
125127,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""PIZ0119"", ""volu...",DPI65908,28.677967,77.104601,28.675770,77.110650,90346559_DPI65908,2021-12-17T11:23:50.239+05:30,2021-12-17T11:03:35+05:30,2021-12-17T11:03:40.845,39820,2021-12-17 11:03:40.845,2021-12-17,11
125128,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""PIZ0134"", ""volu...",DPI65796,28.679921,77.208224,28.674266,77.204407,90321520_DPI65796,2021-12-17T11:24:06.535+05:30,2021-12-17T11:03:05+05:30,2021-12-17T11:09:12.433,40152,2021-12-17 11:09:12.433,2021-12-17,11
125129,2021-12-17,"[{""quantity"": 1, ""menu_code"": ""PIZ0117"", ""volu...",DPI67060,28.506131,77.231868,28.506864,77.240543,90338690_DPI67060,2021-12-17T11:22:34.495+05:30,2021-12-17T11:02:49+05:30,2021-12-17T11:02:58.836,39778,2021-12-17 11:02:58.836,2021-12-17,11


In [21]:
def tx(t):
    try:
        dt = datetime.datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%f")
        return dt
    except:
        try:
            dt = datetime.datetime.strptime(t, "%Y-%m-%dT%H:%M:%S")
            return dt
        except:
            t = t[:-6]
            dt = datetime.datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%f")
            return dt

In [22]:
l = []
for i,x in df.iterrows():
    l.append(tx(x['delivered_time'])-tx(x['order_received_time']))
l
    

[datetime.timedelta(seconds=43113, microseconds=846000),
 datetime.timedelta(seconds=19108, microseconds=930000),
 datetime.timedelta(seconds=1185, microseconds=717000),
 datetime.timedelta(seconds=1692, microseconds=157000),
 datetime.timedelta(seconds=2225, microseconds=418000),
 datetime.timedelta(seconds=1738, microseconds=429000),
 datetime.timedelta(seconds=1378, microseconds=39000),
 datetime.timedelta(seconds=2168, microseconds=890000),
 datetime.timedelta(seconds=833, microseconds=800000),
 datetime.timedelta(seconds=1425, microseconds=91000),
 datetime.timedelta(seconds=1778, microseconds=533000),
 datetime.timedelta(seconds=1104, microseconds=473000),
 datetime.timedelta(seconds=1109, microseconds=159000),
 datetime.timedelta(seconds=1560, microseconds=406000),
 datetime.timedelta(seconds=1693, microseconds=846000),
 datetime.timedelta(seconds=1363, microseconds=167000),
 datetime.timedelta(seconds=1430, microseconds=566000),
 datetime.timedelta(seconds=1693, microseconds=90

In [24]:
len(l), min(l), max(l)

(530,
 datetime.timedelta(seconds=160, microseconds=894000),
 datetime.timedelta(seconds=43113, microseconds=846000))

In [28]:
l2 = [x.total_seconds() for x in l]
l2

[43113.846,
 19108.93,
 1185.717,
 1692.157,
 2225.418,
 1738.429,
 1378.039,
 2168.89,
 833.8,
 1425.091,
 1778.533,
 1104.473,
 1109.159,
 1560.406,
 1693.846,
 1363.167,
 1430.566,
 1693.901,
 1287.975,
 2103.047,
 1560.931,
 1485.03,
 1472.513,
 1193.697,
 1084.276,
 948.631,
 1437.942,
 1040.843,
 1364.496,
 1054.04,
 1039.452,
 1948.876,
 2613.65,
 1949.931,
 1593.621,
 1568.76,
 1015.473,
 2382.821,
 2665.696,
 1580.305,
 1498.635,
 593.831,
 1059.245,
 1312.936,
 1401.183,
 1670.401,
 9570.044,
 7543.764,
 6633.033,
 2973.555,
 2466.945,
 1103.456,
 1774.641,
 1959.237,
 2024.092,
 2366.088,
 2135.725,
 1472.436,
 1432.005,
 1726.949,
 1563.283,
 1465.155,
 1333.718,
 1630.627,
 912.15,
 1649.821,
 1078.323,
 926.017,
 1091.527,
 3164.138,
 3180.386,
 1201.295,
 980.873,
 1265.864,
 1271.418,
 2121.728,
 1639.999,
 1797.567,
 1361.991,
 1670.441,
 1876.219,
 1485.131,
 935.508,
 1552.516,
 2262.124,
 1585.347,
 2033.965,
 1763.124,
 968.498,
 1315.42,
 923.956,
 1420.226,
 868.

In [29]:
ans = sum(l2)/530
ans

1517.7258226415095

In [30]:
l3 = [1 for x in l2 if x>1800]
sum(l3)

69

In [32]:
l = [x for x in l2 if x<1800]
sum(l)/len(l)

1226.407368763557

In [34]:
for x in l2:
    print(x)

43113.846
19108.93
1185.717
1692.157
2225.418
1738.429
1378.039
2168.89
833.8
1425.091
1778.533
1104.473
1109.159
1560.406
1693.846
1363.167
1430.566
1693.901
1287.975
2103.047
1560.931
1485.03
1472.513
1193.697
1084.276
948.631
1437.942
1040.843
1364.496
1054.04
1039.452
1948.876
2613.65
1949.931
1593.621
1568.76
1015.473
2382.821
2665.696
1580.305
1498.635
593.831
1059.245
1312.936
1401.183
1670.401
9570.044
7543.764
6633.033
2973.555
2466.945
1103.456
1774.641
1959.237
2024.092
2366.088
2135.725
1472.436
1432.005
1726.949
1563.283
1465.155
1333.718
1630.627
912.15
1649.821
1078.323
926.017
1091.527
3164.138
3180.386
1201.295
980.873
1265.864
1271.418
2121.728
1639.999
1797.567
1361.991
1670.441
1876.219
1485.131
935.508
1552.516
2262.124
1585.347
2033.965
1763.124
968.498
1315.42
923.956
1420.226
868.666
1250.012
1027.279
1194.528
851.978
1239.137
1146.257
974.606
1047.548
1032.906
918.406
1493.844
1241.926
1514.271
1567.72
1631.077
1662.638
1808.856
1385.86
1094.449
1418.025
1305.1

In [13]:
# df['x_time'] = df['delivered_time'] - df['order_received_time']
# df['x_time']

TypeError: Cannot subtract tz-naive and tz-aware datetime-like objects

In [17]:
l = []
for date in dates:
    order_path = join(tpath,date,'orders')
    rest_path = join(tpath,date,'restaurants')
    rider_path = join(tpath,date,'vehicles')
    
    Path(order_path).mkdir(parents=True, exist_ok=True) 
    Path(rest_path).mkdir(parents=True, exist_ok=True) 
    Path(rider_path).mkdir(parents=True, exist_ok=True) 
    
    for slot in slots:
        tdf = df[(df['date'].astype(str)==date) & (df['slot'] == slot)]
        
        restList = sorted([storetonodemap[x] for x in tdf['store_id'].unique()])
        restListDF = pd.DataFrame(restList)
        restListDF.to_csv(join(rest_path, str(slot)+'.restaurants'), sep=' ', index=False, header=False)
        
        listDrivers = sorted([(x, drivertonodemap[x]) for x in tdf['rider_id'].unique()])
        listDriversDF = pd.DataFrame(listDrivers)
        listDriversDF.to_csv(join(rider_path, str(slot)+'.vehicles'), index=False, header=False, sep=' ')
        
        print(date, slot, len(restList), len(listDrivers))
        l.append([date, slot, len(restList), len(listDrivers)])
#     break


2021-12-17 0 0 0
2021-12-17 1 0 0
2021-12-17 2 0 0
2021-12-17 3 0 0
2021-12-17 4 0 0
2021-12-17 5 0 0
2021-12-17 6 0 0
2021-12-17 7 0 0
2021-12-17 8 0 0
2021-12-17 9 0 0
2021-12-17 10 2 2
2021-12-17 11 158 378
2021-12-17 12 175 701
2021-12-17 13 178 960
2021-12-17 14 180 1068
2021-12-17 15 180 1083
2021-12-17 16 181 1097
2021-12-17 17 181 1193
2021-12-17 18 182 1423
2021-12-17 19 182 1799
2021-12-17 20 182 1900
2021-12-17 21 182 1755
2021-12-17 22 182 1294
2021-12-17 23 142 614
2021-12-18 0 81 286
2021-12-18 1 40 131
2021-12-18 2 29 85
2021-12-18 3 3 4
2021-12-18 4 0 0
2021-12-18 5 0 0
2021-12-18 6 0 0
2021-12-18 7 0 0
2021-12-18 8 0 0
2021-12-18 9 0 0
2021-12-18 10 4 4
2021-12-18 11 163 383
2021-12-18 12 175 720
2021-12-18 13 179 1063
2021-12-18 14 182 1203
2021-12-18 15 183 1256
2021-12-18 16 183 1254
2021-12-18 17 182 1347
2021-12-18 18 182 1644
2021-12-18 19 183 2052
2021-12-18 20 182 2124
2021-12-18 21 182 1986
2021-12-18 22 183 1486
2021-12-18 23 151 714
2021-12-19 0 93 389
2021-

In [16]:
itemsDict = {}
old = (0,0)
j = 0

for index, order in df.iterrows():
    date = str(order['date'])
    slot = order['slot']
    receivedTime = order['time']
    if (date, slot) != old:
        if(old != (0, 0)):
#             if(date != old[0]): break
            fout.close()
        j+=1
#         if(j>=3): break
        print(receivedTime, date, slot)
        order_path = join(tpath, date, 'orders')
        Path(order_path).mkdir(parents=True, exist_ok=True)
        fout = open(join(order_path, str(slot)+'.orders'), 'w')
        old = (date, slot)
        
    cust_node_id = order['node_id']
    rest_node_id = storetonodemap[order['store_id']]
    itemsList = " ".join([str(x) for x in order['menu']])

    fout.write(" ".join([str(receivedTime), str(cust_node_id), str(rest_node_id), itemsList]))
    fout.write("\n")
    
print(receivedTime, old[0], old[1])
fout.close()

39570 2021-12-17 10
39612 2021-12-17 11
43200 2021-12-17 12
46800 2021-12-17 13
50401 2021-12-17 14
54004 2021-12-17 15
57601 2021-12-17 16
61200 2021-12-17 17
64802 2021-12-17 18
68400 2021-12-17 19
72001 2021-12-17 20
75600 2021-12-17 21
79200 2021-12-17 22
82800 2021-12-17 23
0 2021-12-17 23


In [18]:
import json
import time
itemsDict = {}
old = (0,0)
output = []
j = 0
for index, order in df.iterrows():
    date = str(order['date'])
    slot = order['slot']
    receivedTime = order['time']
    if (date, slot) != old:
        if(old != (0, 0)):
            j+=1
            print(receivedTime, old[0], old[1])
            dftmp = pd.DataFrame(output)
            order_path = join(tpath, old[0], 'orders3')
            Path(order_path).mkdir(parents=True, exist_ok=True)
            dftmp.to_csv(join(order_path, str(old[1])+'.csv'), sep=' ', index=False, header=False)
            output = []
        old = (date, slot)
#         if(j>=2): break

    cust_node_id = order['node_id']
    rest_node_id = storetonodemap[order['store_id']]
    itemsList = " ".join([str(x) for x in order['menu']])

    tmp = " ".join([str(receivedTime), str(cust_node_id), str(rest_node_id), itemsList])
    output.append([tmp])
    
print(receivedTime, old[0], old[1])
dftmp = pd.DataFrame(output)
dftmp.to_csv(join(tpath, old[0],'orders', str(old[1])+'.csv'), sep=' ', index=False, header=False)

39612 2021-12-17 10
43200 2021-12-17 11
46800 2021-12-17 12
50401 2021-12-17 13
54004 2021-12-17 14
57601 2021-12-17 15
61200 2021-12-17 16
64802 2021-12-17 17
68400 2021-12-17 18
72001 2021-12-17 19
75600 2021-12-17 20
79200 2021-12-17 21
82800 2021-12-17 22
0 2021-12-17 23
3614 2021-12-18 0
7212 2021-12-18 1
10812 2021-12-18 2
39497 2021-12-18 3
39677 2021-12-18 10
43201 2021-12-18 11
46801 2021-12-18 12
50400 2021-12-18 13
54001 2021-12-18 14
57607 2021-12-18 15
61200 2021-12-18 16
64800 2021-12-18 17
68400 2021-12-18 18
72000 2021-12-18 19
75601 2021-12-18 20
79200 2021-12-18 21
82801 2021-12-18 22
2 2021-12-18 23
3602 2021-12-19 0
7202 2021-12-19 1
10804 2021-12-19 2
39604 2021-12-19 3
43219 2021-12-19 11
46802 2021-12-19 12
50401 2021-12-19 13
54000 2021-12-19 14
57601 2021-12-19 15
61200 2021-12-19 16
64801 2021-12-19 17
68400 2021-12-19 18
72001 2021-12-19 19
75600 2021-12-19 20
79201 2021-12-19 21
82807 2021-12-19 22
4 2021-12-19 23
3602 2021-12-20 0
7200 2021-12-20 1
10802 20

OSError: Cannot save file into a non-existent directory: 'data3/orders/2021-12-31/orders'

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 476882 entries, 123019 to 246791
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype                                
---  ------               --------------   -----                                
 0   order_date           476882 non-null  object                               
 1   order_items          476882 non-null  object                               
 2   store_id             476882 non-null  object                               
 3   store_lat            476882 non-null  float64                              
 4   store_lng            476882 non-null  float64                              
 5   customer_lat         476882 non-null  float64                              
 6   customer_lng         476882 non-null  float64                              
 7   rider_id             476882 non-null  object                               
 8   delivered_time       476882 non-null  datetime64[ns, pytz.FixedOffset

In [ ]:
import json
itemsDict = {}
fileHandlersMap = {}
for order in ordersDelhi:
    receivedTime = int(TimeToEpoch(order['order_received_time']))
    print(order['order_received_time'])
    cust_lat = order['customer_lat']
    cust_lng = order['customer_lng']
    cust_node_id = latLongToNodeId(cust_lat, cust_lng, nodeToLatLongMap)
    rest_node_id = storeToNodeMap[order['store_id']]
    date = order['order_date']
    itemsList = getItemsList(order['order_items'], itemsDict)
    slot = getTimeIndex(order['order_received_time'])
    if((date, slot) in fileHandlersMap):
        fileHandlersMap[(date, slot)].write(str(receivedTime)+","+str(cust_node_id)+","+str(rest_node_id)+",")
        for i in itemsList:
            fileHandlersMap[(date, slot)].write(str(i)+" ")
        fileHandlersMap[(date, slot)].write('\n')
    else:
        f = open(join(tpath, date, slot+'.csv'), 'w')
        fileHandlersMap[(date, slot)] = f
#         fileHandlersMap[(date, slot)].write("order_received_time,cust_node_id,rest_node_id,ordered_items\n")
        fileHandlersMap[(date, slot)].write(str(receivedTime)+" "+str(cust_node_id)+" "+str(rest_node_id)+" ")
        for i in itemsList:
            fileHandlersMap[(date, slot)].write(str(i)+" ")
        fileHandlersMap[(date, slot)].write('\n')  

for k in fileHandlersMap:
    fileHandlersMap[k].close()

In [28]:
import json
import time
itemsDict = {}
old = (0,0)
output = []
j = 0
t1 = t2 = t3 = t4 = 0
for order in ordersDelhi:
    receivedTime = int(TimeToEpoch(order['order_received_time']))
    date = order['order_date']
    slot = getTimeIndex(order['order_received_time'])
    if (date, slot) != old:
        if(old != (0, 0)):
            j+=1
            print(receivedTime, old[0], old[1])
#             dftmp = pd.DataFrame(output)
#             dftmp.to_csv('./orders/'+str(old[0])+'/'+str(old[1])+".csv", sep=',', index=False, header=False)
            output = []
        old = (date, slot)
        if j>=2:
            break

    cust_lat = order['customer_lat']
    cust_lng = order['customer_lng']

    t = time.time()
    cust_node_id = latLongToNodeId(cust_lat, cust_lng, nodeToLatLongMap)
    t1 += time.time() - t
    
    t = time.time()
    rest_node_id = storeToNodeMap[order['store_id']]
    t2 += time.time() - t

    t = time.time()
    itemsList = getItemsList(order['order_items'], itemsDict)
    t3 += time.time() - t

    t = time.time()
    tmp = [receivedTime, cust_node_id, rest_node_id]
    st = ""
    for i in itemsList:
        st += str(i)+" "
    tmp.append(st)
    output.append(tmp)
    t4 += time.time() - t
#     t = time.time()
    
print(receivedTime, old[0], old[1])
# dftmp = pd.DataFrame(output)
# dftmp.to_csv('./orders/'+str(old[0])+'/'+str(old[1])+".csv", sep=',', index=False, header=False)

print(t1,t2,t3,t4)



39612 2021-12-17 10
43200 2021-12-17 11
43200 2021-12-17 12
412.90373182296753 0.016201019287109375 0.10291934013366699 0.013493061065673828


In [2]:
import json
import time
itemsDict = {}
old = (0,0)
output = []
j = 0
print(1,time.time())
for order in ordersDelhi:
    receivedTime = int(TimeToEpoch(order['order_received_time']))
#     print(order['order_received_time'])
    date = order['order_date']
    slot = getTimeIndex(order['order_received_time'])
    if (date, slot) != old:
        if(old != (0, 0)):
            j+=1
            print(receivedTime, old[0], old[1])
            print(1,time.time())
            dftmp = pd.DataFrame(output)
            print(1,time.time())
#             dftmp.to_csv('./orders/'+str(old[0])+'/'+str(old[1])+".csv", sep=' ', index=False, header=False)
            print(1,time.time())
            output = []
        old = (date, slot)
        if j>=2:
            break

    cust_lat = order['customer_lat']
    cust_lng = order['customer_lng']
    cust_node_id = latLongToNodeId(cust_lat, cust_lng, nodeToLatLongMap)
    rest_node_id = storeToNodeMap[order['store_id']]
    itemsList = getItemsList(order['order_items'], itemsDict)
    
    tmp = [receivedTime, cust_node_id, rest_node_id]
#     st = ""
    for i in itemsList:
        tmp.append(i)
#         st += str(i)+" "

#     tmp.append(st)
    output.append(tmp)
    
print(1,time.time())
print(receivedTime, old[0], old[1])
print(1,time.time())
dftmp = pd.DataFrame(output)
print(1,time.time())
# dftmp.to_csv('./orders/'+str(old[0])+'/'+str(old[1])+".csv", sep=',', index=False, header=False)
print(1,time.time())

1 1663707457.4801888


NameError: name 'ordersDelhi' is not defined

In [36]:
output

[]

In [ ]:
import time
time.time()

In [ ]:
### screen starts at 6:50 am